# Test data prep on small sample

In [2]:
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom python")
from silvhua import *


In [29]:
frown = pd.read_csv('./sample_tweets.txt', header=None, usecols=[3], names=['text'])
print(frown.shape)
frown['label'] = 1
frown

(6, 1)


,text,label
0,"""text"": ""RT @JOSHUJIHOUR: i miss this hair :(...",1
1,"""text"": ""RT @tyrannodi_17: pls pls pls if kay...",1
2,"""text"": ""RT @SarahGrimmerTV: Beaches at Kensi...",1
3,"""text"": ""@AnfieldWatch American owners don\u2...",1
4,"""text"": ""anyone has laryngitis before? if so ...",1
5,"""text"": "":( last sequence era limited cards a...",1


# *End of Page*